# CIS Policies of Assets

In [ ]:
import sys
import time, datetime
import pandas as pd
import requests
import urllib3
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)
from IPython.display import display, HTML
from bs4 import BeautifulSoup
import re

from insightvm_authen import InsightvmAuthentication

In [ ]:
HOSTNAME = InsightvmAuthentication().getHostname()
HEADER = InsightvmAuthentication().getRequestHeader()
# HOSTNAME, HEADER

## Get All Assets in Site by SiteID

In [ ]:
def getAssetsInSite(siteID=""):
    assets, page = [], 0
    print("\nGetting Assets of Your Site")
    
    getAssetsInSiteURL = HOSTNAME + "api/3/sites/" + str(siteID) + "/assets"
    
    params = {
        'size': 500,
        'page': 0
    }
    
    assetAllInfos = []
    
    while True:
        print("Hostname : " + getAssetsInSiteURL)
        assetInPage = requests.get(getAssetsInSiteURL, params=params, headers=HEADER, verify=False)
        
        if assetInPage.status_code == 200:
            assetInPage = assetInPage.json()
        
            if assetInPage["page"]["totalResources"] == 0:
                print("[Not Found] Don't have assets in your site ID")
                sys.exit()

            assetAllInfos += assetInPage["resources"]
            
            if (assetInPage["page"]["totalPages"] == params["page"]+1) and (assetInPage["page"]["totalResources"] == len(assetAllInfos)):
                assetAddressInfos = [{"id": ass ["id"], "ip": ass ["ip"], "hostName": ass ["hostName"]} for ass in assetAllInfos]
                return assetAllInfos, assetAddressInfos
            else:
                params["page"] += 1
        else:
            print("[Error] Request Connection Error.")
            print(assetInPage.status_code())
            sys.exit()

In [ ]:
siteIDs = ['1104']

In [ ]:
siteID = siteIDs[0]
assetAllInfos, assetAddressInfos = getAssetsInSite(siteID=siteID)
len(assetAddressInfos), assetAddressInfos

## Get Policies of Asset

In [ ]:
def extractCISPolicy(data={}):
    return {
        "surrogateId": str(data["surrogateId"]),
        "policyID":data["id"],
        "policyName":data["policyName"],
        "ruleCompliance":data["ruleCompliance"],
        "policyTitle":data['title'],
        "benchmarkName":data["benchmarkName"],
        "failedRulesCount": data["failedRulesCount"],
        "passedRulesCount": data["passedRulesCount"]
    }

In [ ]:
def getCISPoliciesofAsset(assetID=None):
    getCISPoliciesofAssetURL = HOSTNAME + 'api/3/assets/' + str(assetID) + '/policies'

    params = {
        'size': 500, # Get limited Policies in Assets : 500
        'page': 0,
        'applicableOnly': True
    }
    
    print("Hostname : " + getCISPoliciesofAssetURL)
    
    CISPoliciesofAsset = requests.get(getCISPoliciesofAssetURL, params=params, headers=HEADER, verify=False)
    CISPoliciesofAsset = CISPoliciesofAsset.json()
    
    if CISPoliciesofAsset["page"]["totalResources"] == 0:
        print("[Error] Not Found CIS Policies.")
        
    if CISPoliciesofAsset["page"]["totalPages"] > 1:
        print("[Error] CIS Policies in Asset more than 500.")
        print("CIS Policies : %d" % (totalResources))
    else:
        print("Getting Successful.")
        return [extractCISPolicy(data=b) for b in CISPoliciesofAsset["resources"]]

In [ ]:
assetID = assetAddressInfos[46]["id"]
policyInAsset = getCISPoliciesofAsset(assetID=assetID)

In [ ]:
pd.DataFrame(policyInAsset)

## Get Policies of Assets

In [ ]:
def getCISPoliciesofAssets(assets=[]):
    policiesInAssets = []
    
    for i, asset in enumerate(assets):
        print("Asset %d of %d Assets" % (i+1, len(assets)))
        policyInAsset = getCISPoliciesofAsset(assetID=asset["id"])
        
        policiesInAssets.append({
            "ip": asset["ip"],
            "assetId": asset["id"],
            "hostName": asset["hostName"],
            "policies": policyInAsset
        })
        
        time.sleep(2)
    return policiesInAssets

In [ ]:
policiesInAssets = getCISPoliciesofAssets(assets=assetAddressInfos)

In [ ]:
pd.DataFrame(policiesInAssets)

## CIS Policies and Assets Matrix Table

Merging and Mapping CISPolicies and Score of Assets in sitescans

In [ ]:
def summaryAssetsInPolicies(policiesInAssets=[]):
    policies = {}
    for asset in policiesInAssets:
        for policy in asset["policies"]:
            if policy["surrogateId"] not in policies:
                policies[policy["surrogateId"]] = {
                    "policyTitle" : policy["policyTitle"],
                    "assets" : [asset["assetId"]]
                }
            else:
                policies[policy["surrogateId"]]["assets"].append(asset["assetId"])
    return policies

In [ ]:
policiesMatrix = summaryAssetsInPolicies(policiesInAssets=policiesInAssets)

In [ ]:
assetIDs = [asset["assetId"] for asset in policiesInAssets]
assetIPs = [asset["ip"] for asset in policiesInAssets]
len(assetIDs), len(assetIPs)

In [ ]:
policiesMxs = []
for k,data in policiesMatrix.items():
    policiesMx = [data["policyTitle"]]
    for ip in assetIDs:
        if ip in data["assets"]: policiesMx.append(0)
        else: policiesMx.append(1)
    
    if 0 not in policiesMx[2:]: 
        print("Match Policies in All Assets: " + data["policyTitle"])

    policiesMxs.append(policiesMx)

In [ ]:
pd.DataFrame(policiesMxs, columns=["Policy"] + assetIPs)

In [ ]:
nowTime = datetime.datetime.now().strftime("(%y%m%d-%H%M%S) ")
df.to_excel ("outputs/" + nowTime + 'policiesMap_' + siteID + '.xlsx', index = False, header=True)

# Get Policy detail by Policy ID

In [ ]:
def getPolicyDetail(policyID=None):
    print("\nGetting Policy Detail")
    r = requests.get(HOSTNAME + "api/3/policies/" + str(policyID), headers=HEADER, verify=False)
    if r.status_code == 200:
        r = r.json()
        return r
        if r["policy"]:
            return r["policy"]
        else:
            print("\n[Not Found] Your Policy in Scan Template")
    else:
        print("[Error] Request Connection Error.")
        print(r.status_code)

In [ ]:
policyId=policiesInAssets[0]["policies"][0]["surrogateId"]

In [ ]:
getPolicyDetail(policyID=policyId)

## Get Rules by AssetId, PolicyID

In [ ]:
def getRuleofAsset(assetID=None, policyId=None):
    # use surrogateId for policyId
    getRuleProofofAssetURL = HOSTNAME + 'api/3/assets/' + str(assetID) + '/policies/' + str(policyId) + '/rules'
    
    print("Asset ID : %d" % (assetID))
    print("Policies ID (SurrogateId) : %s" % (policyId))
    
    params = {
        'page' : 0,
        'size' : 500
    }
    
    rulesofAsset = []
    
    while True:
        print("Hostname : " + getRuleProofofAssetURL)
        
        ruleofAsset = requests.get(getRuleProofofAssetURL, params=params, headers=HEADER, verify=False)

        if ruleofAsset.status_code == 200:
            ruleofAsset = ruleofAsset.json()

            if ruleofAsset["page"]["totalResources"] == 0:
                print("[Not Found] Don't have rules in your AssetID and PolicyID")
                sys.exit()

            rulesofAsset += ruleofAsset["resources"]

            if (ruleofAsset["page"]["totalPages"] == params["page"]+1 and ruleofAsset["page"]["totalResources"] == len(rulesofAsset)):
                return rulesofAsset
            else:
                params["page"] += 1
        else:
            print("[Error] Request Connection Error.")
            print(ruleofAsset.status_code())
            sys.exit()

In [ ]:
# policiesInAssets[0]

In [ ]:
assetID=policiesInAssets[0]["assetId"]
policyId=policiesInAssets[0]["policies"][0]["surrogateId"]

In [ ]:
rulesofAsset = getRuleofAsset(assetID=assetID, policyId=policyId)
len(rulesofAsset)

In [ ]:
rulesofAsset[0]["benchmark"]

In [ ]:
pd.DataFrame(rulesofAsset)

## Get Rule Proof by AssetId, PolicyID, RuleID

In [ ]:
def getRuleProofofAsset(assetID=None, policyId=None, ruleId=None):
    getRuleProofofAssetURL = HOSTNAME + 'api/3/policies/' + str(policyId) + '/rules/' + str(ruleId) + '/assets/' + str(assetID) + '/proof'

    print("Hostname : " + getRuleProofofAssetURL)
    
    header = {
        'Host': HEADER["Host"],
        'Authorization': HEADER["Authorization"]
    }
    
    proofofAsset = requests.get(getRuleProofofAssetURL, headers=header, verify=False)

    if proofofAsset.status_code == 200:
        return proofofAsset.text
    else:
        print("[Error %d] Not found Proof" % (proofofAsset.status_code))

In [ ]:
policy = policiesInAssets[0]

In [ ]:
assetID=policy["assetId"]
policyId=policy["policies"][0]["surrogateId"]

In [ ]:
proofOfAsset = getRuleProofofAsset(assetID=assetID, policyId=policyId, ruleId=rulesofAsset[0]["surrogateId"])
display(HTML(proofOfAsset))

In [ ]:
proofOfAsset

In [ ]:
proofOfAsset_text = BeautifulSoup(proofOfAsset, features="html.parser")
# proofOfAsset_text
# proofOfAsset = re.sub(' +', ' ', proofOfAsset)
# proofOfAsset = re.sub('\n+', '\n', proofOfAsset)
proofOfAsset_text.get_text().splitlines()

## Get Rule Remediate by PolicyID, RuleID

In [ ]:
def getRuleRemediate(policyId=None, ruleId=None):
    getRuleRemediateURL = HOSTNAME + 'api/3/policies/' + str(policyId) + '/rules/' + str(ruleId) + '/remediation'

    print("Hostname : " + getRuleRemediateURL)
    
    header = {
        'Host': HEADER["Host"],
        'Authorization': HEADER["Authorization"]
    }
    
    getRuleRemediate = requests.get(getRuleRemediateURL, headers=header, verify=False)

    if getRuleRemediate.status_code == 200:
        return getRuleRemediate.text
    else:
        print("[Error %d] Not found Proof" % (getRuleRemediate.status_code))

In [ ]:
policy = policiesInAssets[0]
policyId=policy["policies"][0]["surrogateId"]

In [ ]:
ruleRemediate = getRuleRemediate(policyId=policyId, ruleId=rulesofAsset[0]["surrogateId"])
display(HTML(ruleRemediate))

In [ ]:
ruleRemediate_text = BeautifulSoup(ruleRemediate, "lxml").text
ruleRemediate_text = re.sub(' +', ' ', ruleRemediate_text)
ruleRemediate_text = re.sub('\n+', '\n', ruleRemediate_text)
ruleRemediate_text